In [1]:
import numpy as np
import random
import copy
import sys

# ↓問題生成

In [2]:
def puzzle_generation(puzzle_board, issue):
    #ファイル読み込み
    f = open(issue, 'r', encoding='UTF-8')
    data = f.readlines()
    f.close()


    #問題フォーマットの1行目の読み取り ----------------------------------------------------------------------------------------
    s0 = data[0].rstrip('\n').rstrip(',')
    l0 = [int(x.strip()) for x in s0.split(',')]

    colmun = l0[0]              #列
    row = l0[1]                 #行
    selection_cost = l0[2]      #選択コスト


    
    #スライドパズルの盤面の読み取り ------------------------------------------------------------------------------------------
    for i in range(1, row+1):
        row_data = data[i].rstrip('\n').rstrip(',')
        line_data = [int(x.strip()) for x in row_data.split(',')]
        row_array = np.array(line_data)
        if i == 1:
            puzzle_board = row_array
        else:
            puzzle_board = np.vstack((puzzle_board, row_array))

    # colmun = 5
    # row = 6
    #ランダム生成 ---------------------------------------------------------------------------------------------------------
    # random_num = random.sample(range(1, row*colmun+1), row*colmun)
    # random_list = np.array(random_num)
    # puzzle_board = random_list.reshape(row, colmun)


    return puzzle_board, colmun, row, selection_cost

# ↓移動関数

In [3]:
#選択画像を上に移動
def up(target_value, selection_value, exchange_operation):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[0] != 0:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]-1][selection_coord[1]])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]-4], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'U'

        return target_coord, selection_coord, exchange_operation

    
    else:
        return sys.exit("これ以上、上に行けないよ!")

In [4]:
#選択画像を下に移動
def down(target_value, selection_value, exchange_operation):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[0] != 3:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]+1][selection_coord[1]])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]+4], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'D'

        return target_coord, selection_coord, exchange_operation

    else:
        return sys.exit("これ以上、下に行けないよ!")

In [5]:
#選択画像を左に移動
def left(target_value, selection_value, exchange_operation):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[1] != 0:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]][selection_coord[1]-1])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]-1], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'L'

        return target_coord, selection_coord, exchange_operation

    else:
        return sys.exit("これ以上、左に行けないよ!")

In [6]:
#選択画像を右に移動
def right(target_value, selection_value, exchange_operation):
    target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

    if selection_coord[1] != 3:
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]], puzzle_board[selection_coord[0]][selection_coord[1]+1])
        np.put(puzzle_board, [row * selection_coord[0] + selection_coord[1]+1], selection_value)

        target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
        selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))

        exchange_operation += 'R'

        return target_coord, selection_coord, exchange_operation

    else:
        return sys.exit("これ以上、右に行けないよ!")

# ↓回答書き込み

In [7]:
def answer_writing(selection_position, exchange_times, exchange_operation):
    f = open('answer.txt', 'w')

    f.write(str(selection_position[0]) + str(selection_position[1]) + '\n')
    f.write(str(exchange_times) + '\n')
    f.write(exchange_operation + '\n')    

    f.close()

# ↓それぞれの処理

In [8]:
# 2行を除いた盤面の並び替え(たくと)
def puzzle_board_sort_top(puzzle_board, selection_times, exchange_operation):
    #選択値の初期位置を記憶させておく
    selection_value = colmun * row
    selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
    selection_position = selection_coord                #選択画像初期位置

    for i in range(row - 2):
        for j in range(colmun):
            target_value = i * colmun + j + 1           #動かしたい値
            selection_value = colmun * row              #選択値

            target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
            # target_x = target_coord[1]                  #動かしたい値のx座標
            # target_y = target_coord[0]                  #動かしたい値のy座標
            selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
            # selection_x = selection_coord[1]            #選択値のx座標
            # selection_y = selection_coord[0]            #選択値のy座標
            
            up(target_value, selection_value, exchange_operation)
            # down(target_value, selection_value, exchange_operation)
            # left(target_value, selection_value, exchange_operation)
            # right(target_value, selection_value, exchange_operation)

            print(puzzle_board)
            print()

    # return exchange_operation


In [9]:
# 下2行の右4マスを除いた盤面の並び替え(たいち)
def puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_operation):
    print("がんば!")

In [10]:
# 右下4ますの並び替え(しのべー)
def puzzle_board_sort_bottom_right(puzzle_board, selection_times, exchange_operation):
    #選択画像変更次には answer_writing を実行して欲しい

    print("がんば!")


In [11]:
# 上3つをまとめたもの
def puzzle_board_sort(puzzle_board):
    selection_times = 0                     #選択回数
    exchange_operation = ""                 #交換操作

    f = open('answer.txt', 'w')
    f.write('2\n')    
    f.close()

    #たくと担当
    puzzle_board_sort_top(puzzle_board, selection_times, exchange_operation)
    #たいち担当
    # puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_operation)
    # #しのべー担当
    # puzzle_board_sort_bottom_right(puzzle_board, selection_times, exchange_operation)

# メイン関数

In [12]:
puzzle_board =  np.array([])    #現在の盤面を表す配列

#問題盤面生成
puzzle_board, colmun, row, selection_cost = puzzle_generation(puzzle_board, 'issue/4x4.txt')

print('colmun(列→)：' + str(colmun))
print('row(行↓)：' + str(row))
print('selection_cost(選択コスト)：' + str(selection_cost) + '\n')
print(puzzle_board)
print()


#並び替え
puzzle_board_sort(puzzle_board)
print(puzzle_board)

colmun(列→)：4
row(行↓)：4
selection_cost(選択コスト)：2

[[11  9  5  8]
 [12  4  7 10]
 [15  3  1 13]
 [14  2 16  6]]

[[11  9  5  8]
 [12  4  7 10]
 [15  3 16 13]
 [14  2  1  6]]

[[11  9  5  8]
 [12  4 16 10]
 [15  3  7 13]
 [14  2  1  6]]

[[11  9 16  8]
 [12  4  5 10]
 [15  3  7 13]
 [14  2  1  6]]



SystemExit: これ以上、上に行けないよ!